# Load Features

In [1]:
import pickle
X_train = pickle.load(open( 'feat_table/xgb_1_x_train_table.pkl', "rb" ))
y_train = pickle.load(open( 'feat_table/xgb_1_y_train_table.pkl', "rb" ))
X_valid = pickle.load(open( 'feat_table/xgb_1_x_valid_table.pkl', "rb" ))
y_valid = pickle.load(open( 'feat_table/xgb_1_y_valid_table.pkl', "rb" ))
test = pickle.load(open( 'feat_table/xgb_1_test_table.pkl', "rb" ))
features = pickle.load(open( 'feat_table/xgb_1_features_vector_vector.pkl', "rb" ))

In [2]:
# Metrics - not used
import numpy as np
def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

array([ 8.78905071,  9.03574886,  8.93221251, ...,  9.29697668,
        8.55660619,  8.6845703 ])

# LSTM Model basic

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout

Using TensorFlow backend.


### Model Specific Preprocessing

In [9]:
x_train_array = np.copy(X_train[features].values)
x_valid_array = np.copy(X_valid[features].values)

In [10]:
format_sub_train_x = x_train_array.reshape((x_train_array.shape[0], 1, x_train_array.shape[1]) )
format_sub_val_x = x_valid_array.reshape((x_valid_array.shape[0], 1, x_valid_array.shape[1]) )

### Define new loss Function

In [38]:
from keras import backend as K
def mean_squared_percentage_error(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None)), axis=-1)
mspe = MSPE = mean_squared_percentage_error

### Model Definition

In [40]:
model = Sequential()
model.add(LSTM(50, input_shape=(format_sub_train_x.shape[1], format_sub_train_x.shape[2])) )
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

# Train Model

In [17]:
mod_1 = model.fit(format_sub_train_x, np.copy(y_train), epochs=100, batch_size=100, 
                    validation_data=(format_sub_val_x, np.copy(y_valid)), verbose=2, shuffle=False)

Train on 834205 samples, validate on 10133 samples
Epoch 1/100
 - 11s - loss: 0.3129 - val_loss: 0.3150
Epoch 2/100
 - 11s - loss: 0.3137 - val_loss: 0.3104
Epoch 3/100
 - 11s - loss: 0.3129 - val_loss: 0.3102
Epoch 4/100
 - 11s - loss: 0.3134 - val_loss: 0.3200
Epoch 5/100
 - 11s - loss: 0.3130 - val_loss: 0.3106
Epoch 6/100
 - 11s - loss: 0.3135 - val_loss: 0.3141
Epoch 7/100
 - 11s - loss: 0.3129 - val_loss: 0.3106
Epoch 8/100
 - 11s - loss: 0.3130 - val_loss: 0.3225
Epoch 9/100
 - 11s - loss: 0.3130 - val_loss: 0.3156
Epoch 10/100
 - 11s - loss: 0.3129 - val_loss: 0.3106
Epoch 11/100
 - 11s - loss: 0.3127 - val_loss: 0.3169
Epoch 12/100
 - 11s - loss: 0.3132 - val_loss: 0.3150
Epoch 13/100
 - 11s - loss: 0.3129 - val_loss: 0.3124
Epoch 14/100
 - 11s - loss: 0.3130 - val_loss: 0.3125
Epoch 15/100
 - 11s - loss: 0.3129 - val_loss: 0.3150
Epoch 16/100
 - 11s - loss: 0.3126 - val_loss: 0.3103
Epoch 17/100
 - 11s - loss: 0.3129 - val_loss: 0.3151
Epoch 18/100
 - 11s - loss: 0.3122 - val

# Prediction Code

In [26]:
dtest = np.copy(test[features].values)
test_frame = dtest.reshape((dtest.shape[0], 1, dtest.shape[1]) )

In [27]:
test_probs = model.predict(test_frame)
# Make Submission

In [33]:
test_probs


array([ 8.96096992,  8.96096992,  8.96096992, ...,  8.71639442,
        8.71639442,  8.71639442], dtype=float32)

In [34]:
import pandas as pd
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs.flatten())})

In [35]:
result.to_csv("submission/lstm_model1_submission.csv", index=False)